<a href="https://colab.research.google.com/github/hwangjunseong/my-first-website/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9Dteam_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

사용할 라이브러리

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import numpy.linalg as lin

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier


데이터 모으기 , 종속변수 벡터와 독립변수 벡터를 분리하기

In [ ]:
train = pd.read_csv("/content/fashion-mnist_train.csv")
test = pd.read_csv("/content/fashion-mnist_test.csv")



X_train = train.iloc[:,1:].values
y_train = train.iloc[:,0].values
X_test= test.iloc[:,1:].values
y_test=test.iloc[:,0].values



pca를 진행하기전에 표준화 보통 [-3,3]에 값이 위치, 또는 정규화[0, 1] 둘 중 하나 선택

In [ ]:

scaler_minmax=MinMaxScaler()
X_train_std = scaler_minmax.fit_transform(X_train)
X_test_std = scaler_minmax.transform(X_test)

In [ ]:

#scaler = StandardScaler()
#X_train_std = scaler.fit_transform(X_train)
#X_test_std = scaler.transform(X_test)
#print(np.mean(X_train_std))

PCA

In [ ]:
# features = X_train_std.T
# cov_matrix = np.cov(features)
# eigenvalues, eigenvectors = lin.eig(cov_matrix)
# eigen_dict={}
# len_eigenvalues = len(eigenvalues)
# for i in range(len_eigenvalues):
#     key = eigenvalues[i]
#     value = eigenvectors[:, i]
#     eigen_dict[key] = value
# sorted_eigen_tuple = sorted(eigen_dict.items(), key=lambda x: x[0], reverse=True)

# array_U = []
# n = 3 #축소할 차원의 숫자
# for i in range(n):
#     array_U.append(sorted_eigen_tuple[i][1])
# X_reduced= np.dot( array_U,X_train.T)
# print(X_reduced)

In [ ]:

# PCA 모델 생성
pca = PCA(n_components=200)
X_train_reduced = pca.fit_transform(X_train_std)
X_test_reduced = pca.transform(X_test_std)
print(X_train_reduced)

[[ 3.68571599  4.96088434 -0.05065857 ...  0.15857284 -0.06524552
  -0.16985202]
 [-4.37652362  3.86024446 -1.948038   ... -0.1375505   0.01634953
  -0.26044271]
 [ 7.26511378  1.56887786  2.31516324 ...  0.0121788   0.05416366
  -0.02373151]
 ...
 [ 2.18171709  0.45160091 -0.55613746 ... -0.22213121  0.0233864
  -0.16656806]
 [ 1.46950676  6.06358945 -0.6244239  ...  0.11578111  0.11981714
  -0.21023059]
 [-3.57273496  4.7715986  -0.26523304 ... -0.04419707 -0.10601586
  -0.1260086 ]]


모델 선택, 앙상블
각 모델에서 어떤 최적화 알고리즘이 사용되었는지 => ex 미분해서0, 라그랑주 함수

In [47]:


neigh_model = KNeighborsClassifier(n_neighbors=3)
rnf_model = RandomForestClassifier(random_state=42)
svm_model = SVC()

model = VotingClassifier(
    estimators=[('nm', neigh_model), ('rf', rnf_model), ('svc', svm_model)], voting='hard')
model.fit(X_train_reduced, y_train)

KeyboardInterrupt: ignored

test데이터로 예측

In [ ]:

y_pred = model.predict(X_test_reduced)
print(X_test_reduced)
print(y_pred)

혼동행렬 사용하여 정확도 예측

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

예측된 클래스 값을 testResult.txt 파일에 저장

In [ ]:

with open("testResult.txt", "w") as f:
    for index, prediction in enumerate(y_pred):
      index_str = f"{index:05d}" #5자리의 길이로 표시하고, 빈 자리는 0으로 채우는 문자열을 생성
      result = f"{index_str} {int(prediction)}"
      f.write(result + "\n")


In [ ]:
with open("testResult.txt", "r") as f:
    content = f.read()

print(content)